In [1]:
# dart 설치
!pip install opendartreader
import OpenDartReader
api = '1a35aad9876ed05a56911ad073369f81e6c16c4f'
dart = OpenDartReader(api)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import OpenDartReader
from tqdm import tqdm 
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

---

# 연간 부채비율, 영업이익증가율, 매출액증가율, 당기순이익증가율, ROA, ROE
# 추출을 위한 함수구성 
* 부채총계,자산총계,자본총계,영업이익,당기순이익,매출액 추출

In [ ]:
def find_financial_indicators_version2(stock_name, year, indicators):
    
    
    if str(type(dart.find_corp_code(stock_name))) == "<class 'NoneType'>": # NoneType의 경우 pass 
        pass
    else:
        report = dart.finstate(stock_name, year)
        report = report.loc[report['account_nm'].isin(indicators)] # indicators로 계정명에 해당하는 행 추출
        
        report = report.loc[report['account_nm'].isin(indicators)] # indicators로 계정명에 해당하는 행 추출

        # report 정제 ( CFS+OFS | CFS | OFS )
        if sum(report['fs_nm'] =='연결재무제표') != len(indicators):
            report_ = report.loc[report['fs_nm'] == '연결재무제표']
            nonOFS_indic = list(set(indicators) - set(list(report_['account_nm'])))
            temp = report[report['fs_nm'] == '재무제표']
            temp = temp.loc[temp['account_nm'].isin(nonOFS_indic)]
            report = pd.concat([report_,temp])
        else:
            if sum(report['fs_nm'] =='연결재무제표') == len(indicators):
                report = report[report['fs_nm'] == '연결재무제표']
            else:
                report = report[report['fs_nm'] == '재무제표']
    
        data = []
        # for y,m in zip([year, year-1, year-2], ['thstrm_amount','frmtrm_amount','bfefrmtrm_amount']):
        for y,m in zip([year, year-1], ['thstrm_amount','frmtrm_amount']):
            info = [stock_name, y]
            for indic in indicators:
                if sum(report['account_nm'] == indic) >0:
                    value = report.loc[report['account_nm'] == indic , m ].iloc[0]
                else:
                    value = np.nan
                
                info.append(value)
        
            data.append(info)

    return  pd.DataFrame(data, columns = ['기업','연도'] + indicators)

    

In [ ]:
# 데이터프레임 문자열 모두 숫자로 
def str_to_float(value):
    if type(value) == float:
        return value
    elif value == '-':
        return 0
    elif value == "":
        return 0
    else:
        return float(value.replace(',',''))

### 각 주식 종목별 재무지표 계산(전기:2020, 당기:2021)
#### 부채비율, 영업이익증가율, 매출액증가율, 당기순이익증가율, ROA, ROE

In [ ]:
def various_index(df,indicators):
    for indc in indicators:
        df[indc] = df[indc].apply(str_to_float)
    
    # 부채비율
    df['부채비율'] = df['자본총계'] / df['자산총계']

    # 영업이익 증가율 = (당기 영업이익 - 전기 영업이익) / 전기 영업이익
    df.sort_values(by = ['기업','연도'], inplace = True)
    df['영업이익증가율'] = df['영업이익'].diff() / df['영업이익']
    
    # 매출액증가율 = (당기 매출액 - 전기 매출액) / 전기 매출액
    df['매출액증가율'] = df['매출액'].diff() / df['매출액']

    # 당기 순이익증가율 = (당기순이익 - 전기의 당기순이익) / 전기의 당기순이익
    df['당기순이익증가율'] = df['당기순이익'].diff() / df['당기순이익']

    # ROA = (당기순이익 / 자산총계)
    df['ROA'] = df['당기순이익'] / df['자산총계']

    # ROE = (당기순이익/ 평균 자기자본)
    average_equity = df['자본총계'].rolling(2).mean()
    df['ROE'] = df['당기순이익'] / average_equity

    # # 첫해는 당기,전기자료로 계산되는(ex,diff,mean) 경우 지표가 계산되지 않으므로,nan으로 제거 해줘야함
    extra_ind = list(set(list(df.columns.values))-set(['기업','연도']+indicators)) #추가된 columns
    df.reset_index(drop=True,inplace =True)# sort_value로 오름차순으로 정렬된 후 index 초기화
    for i in extra_ind:
        if i != 'ROA'or'부채비율': # 지표중 ROA는 모든 해 계산 가능
            df.loc[df['연도'] == df['연도'][0],i] = np.nan
    return df

## ROI 계산하기
### ROI = 당기순이익/총자본

### 총자본

#### finstate_all을 이용해서 하나의 지표 추출

당기순이익 추출

In [ ]:
# account_nm의 단일 지표값만 호출: ex)당기순이익 // 기말자본은 1개값만 추출됨-> 정확하지 못한 지표 
def finstateall_financial_index(stockname, year, indicators):
    report = dart.finstate_all(stockname, year)
    report = report.loc[report['account_nm'].isin(indicators)]
    data = []
    info =[stockname, year]
    # 하나의 indic에 대해 여러가지 값이 나올 수 있음. ex 기말자본은 여러개 이땐 이 코드로 출력 불가 print함수 부르기 
    for indic in indicators:
        if len(report[report['account_nm'] == indic]) > 1:
            value = 'sum(element({}))>1 지표사용 불가'.format(indic)
            info.append(value)
        else:
            if len(report[report['account_nm'] == indic]) == 1:
                value = report.loc[report['account_nm'] == indic , 'thstrm_amount' ].iloc[0]
            else:
                value = np.nan
            
            info.append(value)

    data.append(info)
    return pd.DataFrame(data, columns = ['기업','연도'] + indicators)

## 실제 네이버금융 코스피 전체에 대한 재무지표 추출

- 부채비율, 매출액 증가율, 당기순이익율, 영업이익율, ROA,ROE

In [ ]:
def total_state( stockname, year):

    report = dart.finstate_all(stockname, year)
    총자본 = ['기말자본','기초자본']
    report = report.loc[report['account_nm'].isin(총자본)]
    # data = [stockname,year] # 총자본만 df형태로 추출
    for idx,st in zip(range(len(총자본)),(총자본)):
        temp = report.loc[report['account_nm'] == st ][['thstrm_amount']]
        for i in range(len(temp)):
            if temp['thstrm_amount'].iloc[i] == "":
                temp['thstrm_amount'].iloc[i] = 0
                
        value = temp.astype(int).sum()
        
        if idx == 0:
            value_total = value[0]
            
        else:
            value_total += value[0]
            
    return value_total.item()

In [ ]:
# 코스피 종목명 코드 각 리스트로 추출

kospi_total_code = [] # 코드만
kospi_total_name = [] # name만
for val in range(1,37):
    URL = 'https://finance.naver.com/sise/sise_market_sum.nhn?&page={}'.format(val)
    resp = requests.get(URL)
    html = resp.text
    soup = BeautifulSoup(html, 'html.parser')
    kospi_chart_html = soup.select('div.box_type_l tbody')[0]
    for item in kospi_chart_html.find_all('a',class_ = 'tltle'):
        kospi_code = item['href'].split('=')[-1]
        kospi_name = item.get_text()
        kospi_total_code.append(kospi_code)
        kospi_total_name.append(kospi_name)
kospi_total = kospi_total_name 

In [ ]:
df = pd.DataFrame(data = [kospi_total_code] + [kospi_total_name]).T
df.columns =['종목코드','종목명']
df.head()

,종목코드,종목명
0,005930,삼성전자
1,373220,LG에너지솔루션
2,000660,SK하이닉스
3,207940,삼성바이오로직스
4,005935,삼성전자우


In [ ]:
# parameter
year = 2021
indicators = ['부채총계','자산총계','자본총계','영업이익','당기순이익','매출액']

#### dart를 이용해서 구할 수 없는 값은 제외
#### 끌어올 값이 finstate, finstate_all을 이용해 가져오는데, 이걸로 가져올 수 없는 주식종목들이 존재 
- case1) str(type(dart.find_corp_code(kospi_total[i]))) == "<class 'NoneType'>
- case2) dart.finstate(kospi_total[i], year) is None
- case3) dart.finstate_all(kospi_total[i], year) is None

*** 문제: 제외하고나니 전체 코스피 수(약 1900개)에서 533개로 가져올 수 있는
주식 수가 현저히 줄어듦

In [ ]:
# dart에서 없는 값 제외 -> 시간이 너무 오래걸려서 txt파일로 저장해둠 
nondata = []
for i in tqdm(range(len(kospi_total))):
    if str(type(dart.find_corp_code(kospi_total[i]))) == "<class 'NoneType'>" or dart.finstate(kospi_total[i], year) is None or dart.finstate_all(kospi_total[i], year) is None: 
        nondata.append(kospi_total[i])

In [ ]:
### 반드시!!!!!!! nondata를 고를 땐 txt 업로드 후 이 코드 사용 !!!!!!!!!!
f = open('nondart_kospi.txt','r')
nondata = f.read().splitlines()

In [ ]:
len(nondata)

1307

In [ ]:
# nondata 파일 저장
f = open("nondart_kospi.txt","w", encoding='UTF-8', newline='')
for num in range(0,len(nondata)):
    f.write("%s\n"% nondata[num])
f.close
from google.colab import files
files.download('nondart_kospi.txt') #다운로드

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
kospi_total = list(set(kospi_total) - set(nondata))

In [ ]:
len(kospi_total) #다트에서 찾지 못하는 자료 제거하니 종목이 현저히 줄어듦 

533

In [ ]:
# 각 코스피별 재무지표(부채비율, 매출액 증가율, 당기순이익율, 영업이익율, ROA,ROE) -> 개느림

for i in tqdm(range(len(kospi_total))):

    base = find_financial_indicators_version2(kospi_total[i], year, indicators)
    index = various_index(base,indicators)
    index.dropna(inplace = True) # 전기 데이터 값(nan값이 존재) 삭제

    #ROI 지표계산
    total_st = total_state(kospi_total[i],year)
    ROI = total_st/base['당기순이익']
    index['ROI'] = ROI

    if kospi_total[i] == kospi_total[0]:
        total_kospi_index = index
        
    else:
        total_kospi_index = pd.concat([total_kospi_index,index],ignore_index = True)
        

display(total_kospi_index)

100%|██████████| 533/533 [20:07<00:00,  2.26s/it]


,기업,연도,부채총계,자산총계,자본총계,영업이익,당기순이익,매출액,부채비율,영업이익증가율,매출액증가율,당기순이익증가율,ROA,ROE,ROI
0,일신방직,2021,2.497100e+11,1.044005e+12,7.942949e+11,6.097049e+10,6.421038e+10,6.010286e+11,0.760815,0.777787,0.226540,0.837526,0.061504,0.084273,70.562185
1,현대건설,2021,1.020797e+13,1.963726e+13,9.429291e+12,7.535040e+11,5.543770e+11,1.806553e+13,0.480173,0.271441,0.060595,0.589274,0.028231,0.060935,90.623938
2,에이프로젠 MED,2021,1.197494e+11,8.241118e+11,7.043624e+11,-2.672314e+09,-1.250035e+10,1.412470e+11,0.854693,2.486046,0.099856,-1.613377,-0.015168,-0.017929,-275.490948
3,경농,2021,1.519764e+11,3.887858e+11,2.368094e+11,1.675899e+10,2.447501e+10,2.702694e+11,0.609100,0.147802,0.081304,0.572646,0.062952,0.109219,53.176008
4,JW중외제약,2021,4.445136e+11,6.330531e+11,1.885395e+11,3.119985e+10,-1.441453e+09,6.065849e+11,0.297826,1.042403,0.097677,-9.597269,-0.002277,-0.007280,-549.454301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516,ESR켄달스퀘어리츠,2021,7.387885e+11,1.443146e+12,7.043571e+11,1.473025e+10,9.485404e+09,3.006267e+10,0.488071,0.449872,0.211313,0.429729,0.006573,0.013407,447.173208
517,SIMPAC,2021,3.565673e+11,9.032296e+11,5.466623e+11,1.064015e+11,9.083286e+10,6.130128e+11,0.605231,0.926811,0.403172,0.624151,0.100565,0.179752,33.379388
518,락앤락,2021,1.357731e+11,8.297246e+11,6.939515e+11,3.251891e+10,1.615571e+10,5.430375e+11,0.836364,0.110429,0.075598,0.272359,0.019471,0.023771,252.388456
519,기아,2021,3.193744e+13,6.685000e+13,3.491256e+13,5.065685e+12,4.760311e+12,6.986237e+13,0.522252,0.592068,0.153076,0.687503,0.071209,0.146914,40.839886


##### dart에 있는 값들에 대해선 제대로 추출됨.